In [19]:
import tensorflow as tf 

In [20]:
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model


In [21]:
from ampligraph.discovery import find_clusters
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

ModuleNotFoundError: No module named 'matplotlib'

In [22]:
def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


In [23]:
import pandas as pd

dataset_new = pd.read_csv("../data/routes.csv")
dataset_new.columns = ['_start', '_end', '_type','_distance']
dataset_new = dataset_new.reindex(columns=['_start', '_type', '_end', '_distance'])
dataset_new.drop(dataset_new.index[dataset_new['_start'] == '_start'], inplace=True)
dataset_new.drop(columns=['_distance'], inplace=True)
dataset_new.head(10)

,_start,_type,_end
0,8627,HAS_ROUTE,8629
1,8627,HAS_ROUTE,8630
2,8627,HAS_ROUTE,8631
3,8627,HAS_ROUTE,8632
4,8627,HAS_ROUTE,8633
5,8627,HAS_ROUTE,8634
6,8627,HAS_ROUTE,8635
7,8627,HAS_ROUTE,8636
8,8627,HAS_ROUTE,8637
9,8627,HAS_ROUTE,8638


In [24]:
ent_to_id ,rel_to_idx = ampligraph.evaluation.create_mappings(dataset_new.to_numpy())

In [25]:
ampligraph.evaluation.to_idx(dataset_new.to_numpy(), rel_to_idx, ent_to_id)

array([[   0,    0,    2],
       [   0,    0,    3],
       [   0,    0,    4],
       ...,
       [8386,    4, 8623],
       [8387,    4, 8622],
       [8388,    4, 8622]])

In [26]:
print('Total triples in the KG:', dataset_new.shape)

Total triples in the KG: (73954, 3)


In [27]:
from ampligraph.evaluation import train_test_split_no_unseen
# get the validation set of size 500
test_train, X_valid = train_test_split_no_unseen(dataset_new.values, 1000, seed=0)

# get the test set of size 1000 from the remaining triples
X_train, X_test = train_test_split_no_unseen(test_train, 2000, seed=0)

print('Total triples:', dataset_new.shape)
print('Size of train:', X_train.shape)
print('Size of valid:', X_valid.shape)
print('Size of test:', X_test.shape)

Total triples: (73954, 3)
Size of train: (70954, 3)
Size of valid: (1000, 3)
Size of test: (2000, 3)


In [28]:
model = TransE(k=200,                                                             # embedding size
               epochs=300,                                                        # Num of epochs
               batches_count= 10,                                                 # Number of batches 
               eta=2,                                                             # number of corruptions to generate during training
               loss='pairwise', loss_params={'margin': 1},                        # loss type and it's hyperparameters         
               initializer='xavier', initializer_params={'uniform': False},       # initializer type and it's hyperparameters
               regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},   # regularizer along with its hyperparameters
               optimizer= 'adam', optimizer_params= {'lr': 0.001},                # optimizer to use along with its 
               
               
               
               
               
               
               
               
               
               seed= 0, verbose=True)

model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/TransE.pkl')
display_aggregate_metrics(ranks)






100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [00:26<00:00, 75.67it/s]

Mean Rank: 163.081
Mean Reciprocal Rank: 0.18552176422303676
Hits@1: 0.0005
Hits@10: 0.542
Hits@100: 0.8315


In [31]:
model = ComplEx(k=200, epochs=300, eta=1, loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},

                
                
                
                seed= 0, batches_count= 1, verbose=True)
model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/ComplEx.pkl')
display_aggregate_metrics(ranks)


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [01:56<00:00, 17.20it/s]

Mean Rank: 213.251
Mean Reciprocal Rank: 0.2387062421129227
Hits@1: 0.1735
Hits@10: 0.3635
Hits@100: 0.67525


In [32]:
model = DistMult(k=200, epochs=300, eta=1, loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},

                
                
                
                seed= 0, batches_count= 1, verbose=True)
model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/DistMult.pkl')
display_aggregate_metrics(ranks)


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [00:22<00:00, 87.16it/s]

Mean Rank: 230.1245
Mean Reciprocal Rank: 0.2541328064527293
Hits@1: 0.191
Hits@10: 0.3725
Hits@100: 0.65575


In [34]:
model = ConvKB(k=200, epochs=100, eta=1, loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},
                optimizer= 'adam', optimizer_params= {'lr': 0.001}, 
                seed= 0, 
                batches_count= 5, # Goes OOM (ResourceExhaustedError) if batch count is 1
                verbose=True)
model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/ConvKB.pkl')
display_aggregate_metrics(ranks)

100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [25:24<00:00,  1.31it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'Models_old/ConvKB.pkl'

In [35]:
save_model(model, 'Models_new/ConvKB.pkl')
display_aggregate_metrics(ranks)

Mean Rank: 236.736
Mean Reciprocal Rank: 0.22788252786336363
Hits@1: 0.13525
Hits@10: 0.405
Hits@100: 0.74075
